<a href="https://colab.research.google.com/github/palakagl/NLP/blob/main/Wysdom_Distillbert_ZeroShot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unidecode

     |████████████████████████████████| 235 kB 10.9 MB/s 


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import numpy as np
import io

import tensorflow as tf

!pip install -q transformers tfds-nightly
try: # this is only working on the 2nd try in colab :)
  from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer
except Exception as err: # so we catch the error and import it again
  from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords  
from bs4 import BeautifulSoup
import re
import unicodedata
import unidecode
import nltk
from nltk import word_tokenize

from sklearn.metrics.cluster import adjusted_rand_score, adjusted_mutual_info_score

     |████████████████████████████████| 4.0 MB 6.9 MB/s 
     |████████████████████████████████| 4.3 MB 55.9 MB/s 
     |████████████████████████████████| 895 kB 53.0 MB/s 
     |████████████████████████████████| 596 kB 11.3 MB/s 
     |████████████████████████████████| 6.5 MB 55.1 MB/s 
     |████████████████████████████████| 77 kB 7.6 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [3]:
nltk.download("wordnet")
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from google.colab import files
uploaded = files.upload()

Saving personal_assistant_train.csv to personal_assistant_train.csv


In [5]:
df = pd.read_csv(io.BytesIO(uploaded['personal_assistant_train.csv']))
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8884 entries, 0 to 8883
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       8884 non-null   int64 
 1   message  8884 non-null   object
 2   label    8884 non-null   object
dtypes: int64(1), object(2)
memory usage: 208.3+ KB
None
      id                                            message             label
0  10480                           start the robotic vacuum      iot_cleaning
1   4210          i am sorry but i think you are not right.    general_negate
2   2443  what is the time difference between california...  datetime_convert
3   5869                              tell me my list names       lists_query
4   2801                 beep when i get an email from john       email_query


##Preprocessing

In [ ]:
def ds_preprocess(data_set):
  # drop duplicate rows
  data_set = data_set.drop_duplicates()
  # drop NaN data and reset the index 
  data_set=data_set.dropna()
  data_set=data_set.reset_index(drop=True)
  return data_set

def text_preprocess(m_text, lem=False,stem=False):

  # Lemmatization
  if lem == True:
    #df['message'] = [WordNetLemmatizer().lemmatize(w) for w in df['message']]
    m_text = [WordNetLemmatizer().lemmatize(w) for w in m_text]

  #Stemming
  if stem == True: 
    #df['message'] =[SnowballStemmer('english').stem(w) for w in df['message']]
    m_text =[SnowballStemmer('english').stem(w) for w in m_text]

  return m_text

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = re.sub(r'@\w+', '',w)
    return w

df = ds_preprocess(df)
df['message'] = df['message'].map(preprocess_sentence)

stop_words = stopwords.words('english') 
lemmer = WordNetLemmatizer()

def preprocess(x):
    # Remove HTML tags
    x = BeautifulSoup(x, "lxml").get_text()

    # Lower case
    x = x.lower()
    
    # Remove punctuation
    x = re.sub(r'[^\w\s]', '', x)
    
    # Remove non-unicode
    x = unidecode.unidecode(x)
    
    # Remove numbers
    x = re.sub(r'\d+', '', x)
    
    # Remove stopwords and lemmatize
    x = [lemmer.lemmatize(w) for w in x.split() if w not in stop_words]
    return ' '.join(x) 

#%time df['message'] = df['message'].apply(preprocess)
#df.head(15)

#df['message'] = text_preprocess(df['message'], lem=True,stem=False)

In [6]:
df.head(15)

,id,message,label
0,10480,start the robotic vacuum,iot_cleaning
1,4210,i am sorry but i think you are not right.,general_negate
2,2443,what is the time difference between california...,datetime_convert
3,5869,tell me my list names,lists_query
4,2801,beep when i get an email from john,email_query
5,7384,find me the local npr station,play_radio
6,3244,yes answer was confirmed.,general_affirm
7,3247,answer is confirmed.,general_affirm
8,5220,i would love a less bright up room,iot_hue_lightdim
9,6400,i've heard this riff before what song stole th...,music_query


In [7]:
# Encode the labels
df['encoded_label'] = df['label'].astype('category').cat.codes
b = df['label'].astype('category')

#Create the category mapping to map it back
cat_map = dict(enumerate(b.cat.categories))
df.head()

,id,message,label,encoded_label
0,10480,start the robotic vacuum,iot_cleaning,26
1,4210,i am sorry but i think you are not right.,general_negate,22
2,2443,what is the time difference between california...,datetime_convert,10
3,5869,tell me my list names,lists_query,36
4,2801,beep when i get an email from john,email_query,13


## Train test Split

In [8]:
X = df['message'].to_list()
y = df['encoded_label'].to_list()

X_train, X_val, y_train ,y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
def encode_text(text):
    max_length= 28
    dbert_inputs = tokenizer(text,max_length =max_length,
                            truncation=True, 
                            add_special_tokens=True, 
                            padding=True)


    return dbert_inputs
train_encodings = encode_text(X_train)   
val_encodings = encode_text(X_val)

In [10]:
def create_dataset(encoding, label):
    dataset = tf.data.Dataset.from_tensor_slices((
    dict(encoding),
    label))

    return dataset

train_dataset = create_dataset(train_encodings, y_train)
val_dataset = create_dataset(val_encodings, y_val)

In [11]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=64)

optimizer = tf.keras.optimizers.RMSprop(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics= ['accuracy'])

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [12]:
model.fit(train_dataset.shuffle(len(train_dataset)).batch(128), epochs=15, batch_size=128,
          validation_data=val_dataset.shuffle(len(val_dataset)).batch(128))

Epoch 1/15
56/56 [==============================] - 34s 262ms/step - loss: 3.6538 - accuracy: 0.3854 - val_loss: 3.0223 - val_accuracy: 0.6466
Epoch 2/15
56/56 [==============================] - 13s 237ms/step - loss: 2.6150 - accuracy: 0.6962 - val_loss: 2.0608 - val_accuracy: 0.7434
Epoch 3/15
56/56 [==============================] - 13s 237ms/step - loss: 1.7627 - accuracy: 0.7873 - val_loss: 1.3416 - val_accuracy: 0.8019
Epoch 4/15
56/56 [==============================] - 13s 237ms/step - loss: 1.1342 - accuracy: 0.8520 - val_loss: 0.8795 - val_accuracy: 0.8481
Epoch 5/15
56/56 [==============================] - 13s 239ms/step - loss: 0.7094 - accuracy: 0.9052 - val_loss: 0.6117 - val_accuracy: 0.8880
Epoch 6/15
56/56 [==============================] - 13s 238ms/step - loss: 0.4372 - accuracy: 0.9405 - val_loss: 0.4608 - val_accuracy: 0.8998
Epoch 7/15
56/56 [==============================] - 13s 237ms/step - loss: 0.2725 - accuracy: 0.9637 - val_loss: 0.4057 - val_accuracy: 0.9038

In [13]:
output_val = model.predict(val_dataset)
pred_labels_val = np.argmax(output_val.logits, axis=1)

In [14]:
pred_labels_val

array([17, 41, 60, ..., 39, 54, 62])

In [15]:
ari_val = adjusted_rand_score(y_val, pred_labels_val)
ami_val = adjusted_mutual_info_score(y_val, pred_labels_val, average_method='arithmetic')

print("ARI: {}".format(ari_val))
print("AMI: {}".format(ami_val))

ARI: 0.8351046862723964
AMI: 0.889504756138387


## Codalab Predictions

In [16]:
from google.colab import files
uploaded = files.upload()

Saving personal_assistant_test.csv to personal_assistant_test.csv


In [17]:
df_test = pd.read_csv(io.BytesIO(uploaded['personal_assistant_test.csv']))
print(df_test.info())
print(df_test.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2221 entries, 0 to 2220
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       2221 non-null   int64 
 1   message  2221 non-null   object
dtypes: int64(1), object(1)
memory usage: 34.8+ KB
None
      id                                     message
0  10873  list the current price of microsoft shares
1   8491             where is the nearest smoke shop
2   4549                          please start over.
3   5948                         read my lists to me
4   1663                             mute volume now


In [ ]:
df_test = ds_preprocess(df_test)
# Preprossing if needed
df_test['message'] = df_test['message'].map(preprocess_sentence)
# Lemmatization and stemming if needed
#df['message'] = text_preprocess(df['message'], lem=True,stem=False)

In [18]:
pred_test = df_test['message'].to_list()
text_encodings = encode_text(pred_test)
text_ds = tf.data.Dataset.from_tensor_slices((
    dict(text_encodings)))
output = model.predict(text_ds)
pred_labels = np.argmax(output.logits, axis=1)
pred_labels

array([51, 53, 25, ..., 30, 13, 49])

In [19]:
my_submission = pd.DataFrame({'Id': df_test['id'], 'label': pred_labels})
my_submission['label'] = my_submission['label'].map(cat_map)
print(my_submission.head())

# NOTE: after saving the CSV file, be sure to zip the file before submitting to the competition website!
my_submission.to_csv('answers_withoutpreprocess_distillbert.csv', index=False)

      Id                     label
0  10873                  qa_stock
1   8491  recommendation_locations
2   4549            general_repeat
3   5948               lists_query
4   1663         audio_volume_mute
